In [1]:
# Cybersecurity Disinformation Detection Briefing System (CDDBS) - Refactored

# A disinformation detection briefing system

# System Architecture: Fetch -> Analyse -> Digest -> Translate -> Summarize

# Tech Stack: Crawling -> SERPAPI, DB -> Postgres, Analysis -> gemini-2.5-flash

# Workflow Orchestration -> Langgraph

# **1. INSTALLATION AND IMPORTS**

In [2]:
import subprocess
import sys

In [3]:
def install_packages():
  """
  Install required packages for the system
  """
  packages = [
  'langchain-community', 'langgraph', 'langchain-google-genai',
  'google-genai', 'psycopg2-binary', 'google-generativeai',
  'google-search-results', 'requests', 'beautifulsoup4'
  ]

  for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        print(f"✓ Installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to install {package}: {e}")

In [4]:
install_packages()

✓ Installed langchain-community
✓ Installed langgraph
✓ Installed langchain-google-genai
✓ Installed google-genai
✓ Installed psycopg2-binary
✓ Installed google-generativeai
✓ Installed google-search-results
✓ Installed requests
✓ Installed beautifulsoup4


**Core imports**

In [5]:
import os
import json
import psycopg2
import requests
from bs4 import BeautifulSoup
from google import genai
from google.genai import types
from serpapi import GoogleSearch
from langgraph.graph import START, StateGraph, END
from typing import TypedDict, List, Dict, Optional
from IPython.display import Markdown, Image
import time
from datetime import datetime
import logging

# **2. CONFIGURATION AND SETUP**

In [6]:
class CDDBSConfig:
    # Configuration management for CDDBS

  def __init__(self):
    self.setup_logging()
    self.setup_api_keys()
    self.setup_gemini()

  def setup_logging(self):
    """
		Configure logging for the system
		"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    self.logger = logging.getLogger('CDDBS')

  def setup_api_keys(self):
    """
		Setup API keys from environment or user input
		"""
    # Try to get from environment first
    self.gemini_api_key = os.getenv('GOOGLE_API_KEY')
    self.serper_api_key = os.getenv('SERPER_API')

    # If running in Colab, try userdata
    try:
        from google.colab import userdata
        if not self.gemini_api_key:
            self.gemini_api_key = userdata.get('GOOGLE_API_KEY')
        if not self.serper_api_key:
            self.serper_api_key = userdata.get('SERPER_API')
    except ImportError:
        pass

    # Prompt for keys if not found
    if not self.gemini_api_key:
        self.gemini_api_key = input("Enter your Google Gemini API key: ")
    if not self.serper_api_key:
        self.serper_api_key = input("Enter your SerpAPI key: ")

    # Set environment variables
    os.environ['GOOGLE_API_KEY'] = self.gemini_api_key
    os.environ['SERPER_API'] = self.serper_api_key

  def setup_gemini(self):
    """
		Initialize Gemini client
		"""
    try:
        self.client = genai.Client()
        self.default_model = "gemini-2.5-flash"
        self.logger.info("✓ Gemini client initialized")
    except Exception as e:
        self.logger.error(f"✗ Failed to initialize Gemini: {e}")
        raise

In [7]:
# Initialize configuration

config = CDDBSConfig()


# **3. SYSTEM PROMPT AND STATE DEFINITION**

In [8]:
INTELLIGENCE_SYSTEM_PROMPT = """
You are an intelligence analyst specializing in DISINFORMATION DETECTION and cybersecurity RISK REPORTING.
Your task is to create OBJECTIVE, FACT-BASED briefing reports from news articles.

STRICT RULES:
1. DO NOT invent claims, actors, or events.
2. ALWAYS attribute statements to their source (e.g., "According to Xinhua...").
3. Clearly separate facts (what the outlet published) from analysis (disinformation framing, propaganda, sentiment).
4. Use neutral, professional language (no creative writing).
5. IF an article contains unverifiable claims, mark them explicitly as "claim by [outlet]" or "unverified".

OUTPUT FORMAT:
1. Outlet and Source URL (Where information comes from)
2. Main Narrative/Claims (quoted or paraphrased with attribution)
3. Analysis (possible propaganda/disinformation pattern, tone)
4. Credibility notes (confidence, any missing sources, cross-check needs)

The goal is to generate a CREDIBLE analyst briefing that other professionals can rely on.
DO NOT speculate. DO NOT embellish. Stick to cited sources.
"""

In [9]:
class BriefingState(TypedDict):
    """State management for briefing workflow"""
    outlet: str
    url: str
    country: str
    articles: List[Dict]
    summaries: List[str]
    final_report: str
    errors: List[str]
    processing_time: float

# **4. ENHANCED CONTENT EXTRACTION UTILITIES**

In [12]:
class ContentExtractor:
    """Enhanced content extraction with full article fetching"""

    def __init__(self, config):
        self.config = config
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def extract_full_article(self, url: str) -> Optional[str]:
        """Extract full article content from URL"""
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()

            # Try common article selectors
            selectors = [
                'article',
                '.article-content',
                '.post-content',
                '.entry-content',
                '[class*="content"]',
                'main',
                '.main-content'
            ]

            for selector in selectors:
                content = soup.select_one(selector)
                if content and len(content.get_text().strip()) > 200:
                    return content.get_text().strip()

            # Fallback to body text
            body = soup.find('body')
            if body:
                return body.get_text().strip()

            return None

        except Exception as e:
            config.logger.warning(f"Failed to extract content from {url}: {e}")
            return None


# **5. DATABASE UTILITIES (ENHANCED)**

In [10]:
class DatabaseManager:
    """Enhanced database management with error handling"""

    def __init__(self, config):
        self.config = config
        self.connection_params = {
            "dbname": "knowledge",
            "user": "admin",
            "password": "admin",
            "host": "127.0.0.1",
            "port": "5432"
        }

    def get_connection(self):
        """Get database connection with error handling"""
        try:
            return psycopg2.connect(**self.connection_params)
        except psycopg2.Error as e:
            self.config.logger.error(f"Database connection failed: {e}")
            return None

    def save_article(self, outlet: str, url: str, title: str, text: str, lang: str = "en") -> Optional[int]:
        """Save article with proper error handling"""
        conn = self.get_connection()
        if not conn:
            return None

        try:
            cur = conn.cursor()
            cur.execute(
                """INSERT INTO articles (outlet_id, url, title, text, language, created_at)
                   VALUES ((SELECT id FROM outlets WHERE name=%s), %s, %s, %s, %s, %s)
                   RETURNING id""",
                (outlet, url, title, text, lang, datetime.now())
            )
            article_id = cur.fetchone()[0]
            conn.commit()
            cur.close()
            conn.close()
            return article_id
        except psycopg2.Error as e:
            self.config.logger.error(f"Failed to save article: {e}")
            if conn:
                conn.rollback()
                conn.close()
            return None

    def create_tables(self):
        """Create necessary database tables"""
        conn = self.get_connection()
        if not conn:
            return False

        try:
            cur = conn.cursor()

            # Create outlets table
            cur.execute("""
                CREATE TABLE IF NOT EXISTS outlets (
                    id SERIAL PRIMARY KEY,
                    name VARCHAR(255) UNIQUE NOT NULL,
                    url VARCHAR(255) NOT NULL,
                    country VARCHAR(100),
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)

            # Create articles table
            cur.execute("""
                CREATE TABLE IF NOT EXISTS articles (
                    id SERIAL PRIMARY KEY,
                    outlet_id INTEGER REFERENCES outlets(id),
                    url VARCHAR(512) UNIQUE NOT NULL,
                    title TEXT NOT NULL,
                    text TEXT,
                    language VARCHAR(10) DEFAULT 'en',
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)

            # Create analyses table
            cur.execute("""
                CREATE TABLE IF NOT EXISTS analyses (
                    id SERIAL PRIMARY KEY,
                    article_id INTEGER REFERENCES articles(id),
                    analysis_type VARCHAR(50),
                    result TEXT,
                    confidence_score FLOAT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)

            conn.commit()
            cur.close()
            conn.close()
            self.config.logger.info("✓ Database tables created successfully")
            return True

        except psycopg2.Error as e:
            self.config.logger.error(f"Failed to create tables: {e}")
            if conn:
                conn.rollback()
                conn.close()
            return False


In [13]:
# Initialize utilities
content_extractor = ContentExtractor(config)
db_manager = DatabaseManager(config)


# **6. ENHANCED PROCESSING NODES**

In [14]:
def fetch_articles(state: BriefingState, top_n: int = 5) -> BriefingState:
    """Article fetching with full content extraction"""
    config.logger.info(f"Fetching articles from {state['outlet']} ({state['url']})")

    try:
        params = {
            "engine": "google_news",
            "q": f"site:{state['url']}",
            "hl": "en",
            "gl": "us",
            "num": top_n,
            "api_key": config.serper_api_key
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "error" in results:
            state["errors"].append(f"SerpAPI error: {results['error']}")
            return state

        articles = []

        for news in results.get("news_results", [])[:top_n]:
            article = {
                "title": news.get("title", ""),
                "link": news.get("link", ""),
                "snippet": news.get("snippet", ""),
                "date": news.get("date", ""),
                "source": news.get("source", ""),
                "meta": news
            }

            # Try to extract full content
            full_content = content_extractor.extract_full_article(article["link"])
            if full_content:
                article["full_text"] = full_content[:5000]  # Limit length
                config.logger.info(f"✓ Extracted full content for: {article['title'][:50]}...")
            else:
                article["full_text"] = article["snippet"]
                config.logger.warning(f"✗ Using snippet only for: {article['title'][:50]}...")

            articles.append(article)
            time.sleep(1)  # Rate limiting

        state["articles"] = articles
        config.logger.info(f"✓ Fetched {len(articles)} articles")

    except Exception as e:
        error_msg = f"Error fetching articles: {str(e)}"
        config.logger.error(error_msg)
        state["errors"].append(error_msg)

    return state

In [15]:
def analyze_articles(state: BriefingState) -> BriefingState:
    """Analysis with better error handling and retry logic"""
    config.logger.info("Analyzing articles for disinformation patterns")

    analyzed = []

    for i, article in enumerate(state["articles"], 1):
        try:
            content_to_analyze = article.get("full_text", article["snippet"])

            prompt = f"""
            Analyze this news piece for disinformation patterns and propaganda techniques.

            Source: {state['outlet']} ({state['country']})
            Title: {article['title']}
            Date: {article.get('date', 'Unknown')}
            Content: {content_to_analyze[:2000]}

            Provide analysis on:
            1. Narrative type and framing
            2. Propaganda score (0-1, where 1 is highest propaganda)
            3. Emotional manipulation techniques
            4. Factual accuracy concerns
            5. Source credibility indicators
            """

            response = config.client.models.generate_content(
                model=config.default_model,
                config=types.GenerateContentConfig(
                    system_instruction=INTELLIGENCE_SYSTEM_PROMPT,
                    temperature=0.1,
                    max_output_tokens=1000
                ),
                contents=prompt
            )

            article['analysis'] = response.text
            analyzed.append(article)

            config.logger.info(f"✓ Analyzed article {i}/{len(state['articles'])}")
            time.sleep(2)  # Rate limiting for API

        except Exception as e:
            error_msg = f"Error analyzing article '{article['title']}': {str(e)}"
            config.logger.error(error_msg)
            state["errors"].append(error_msg)

            # Add article without analysis
            article['analysis'] = "Analysis failed due to technical error"
            analyzed.append(article)

    state["articles"] = analyzed
    return state


In [16]:
def digest_articles(state: BriefingState) -> BriefingState:
    """Enhanced content digestion with structured extraction"""
    config.logger.info("Extracting key claims and actors")

    for i, article in enumerate(state["articles"], 1):
        try:
            content = article.get("full_text", article["snippet"])

            prompt = f"""
            Extract structured information from this news article:

            Title: {article['title']}
            Content: {content[:2000]}

            Extract:
            1. KEY CLAIMS: List main factual claims made
            2. KEY ACTORS: People, organizations, countries mentioned
            3. NARRATIVE THEMES: Main story angles and framing
            4. UNVERIFIED STATEMENTS: Claims that need fact-checking
            5. EMOTIONAL LANGUAGE: Words/phrases designed to evoke emotion

            Format as structured data for analysis.
            """

            response = config.client.models.generate_content(
                model=config.default_model,
                config=types.GenerateContentConfig(temperature=0.1),
                contents=prompt
            )

            article["digest"] = response.text
            config.logger.info(f"✓ Digested article {i}/{len(state['articles'])}")
            time.sleep(2)

        except Exception as e:
            error_msg = f"Error digesting article '{article['title']}': {str(e)}"
            config.logger.error(error_msg)
            state["errors"].append(error_msg)
            article["digest"] = "Digest generation failed"

    return state

In [17]:
def translate_content(state: BriefingState) -> BriefingState:
    """Enhanced translation with language detection"""
    config.logger.info("Processing translations")

    for i, article in enumerate(state["articles"], 1):
        try:
            # Simple language detection based on content
            content = article.get("digest", "")

            if not content or len(content.strip()) == 0:
                article["translated_digest"] = "No content to translate"
                continue

            prompt = f"""
            Review the following content and ensure it's in clear, professional English.
            If it's already in English, improve clarity and structure.
            If it's in another language, translate to English while preserving meaning.

            Content: {content}

            Provide the final English version with improved clarity and structure.
            """

            response = config.client.models.generate_content(
                model=config.default_model,
                config=types.GenerateContentConfig(temperature=0.1),
                contents=prompt
            )

            article["translated_digest"] = response.text
            config.logger.info(f"✓ Processed translation {i}/{len(state['articles'])}")
            time.sleep(2)

        except Exception as e:
            error_msg = f"Error translating article '{article['title']}': {str(e)}"
            config.logger.error(error_msg)
            state["errors"].append(error_msg)
            article["translated_digest"] = article.get("digest", "Translation failed")

    return state

In [19]:
def generate_intelligence_brief(state: BriefingState) -> BriefingState:
    """Generate final intelligence briefing with enhanced structure"""
    config.logger.info("Generating intelligence briefing")

    try:
        # Compile all analysis data
        briefing_data = {
            "outlet": state['outlet'],
            "country": state['country'],
            "url": state['url'],
            "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "articles_analyzed": len(state['articles']),
            "errors_encountered": len(state.get('errors', []))
        }

        summaries = []

        for i, article in enumerate(state["articles"], 1):
            try:
                prompt = f"""
                Generate a professional intelligence briefing section for this article:

                ARTICLE {i}/{len(state['articles'])}:
                Outlet: {state['outlet']} ({state['country']})
                Title: {article['title']}
                Date: {article.get('date', 'Unknown')}
                Source URL: {article['link']}

                Analysis Data:
                {article.get('analysis', 'No analysis available')}

                Key Information:
                {article.get('translated_digest', 'No digest available')}

                Generate a structured intelligence brief following professional standards:
                1. Source Attribution
                2. Key Claims (with proper attribution)
                3. Disinformation Analysis
                4. Credibility Assessment
                5. Recommended Actions
                """

                response = config.client.models.generate_content(
                    model=config.default_model,
                    config=types.GenerateContentConfig(
                        system_instruction=INTELLIGENCE_SYSTEM_PROMPT,
                        temperature=0.1,
                        max_output_tokens=1500
                    ),
                    contents=prompt
                )

                summaries.append(response.text)
                config.logger.info(f"✓ Generated brief section {i}/{len(state['articles'])}")
                time.sleep(2)

            except Exception as e:
                error_msg = f"Error generating brief for article '{article['title']}': {str(e)}"
                config.logger.error(error_msg)
                state["errors"].append(error_msg)
                summaries.append(f"**Brief Generation Failed for Article {i}**\nError: {str(e)}")

        # Create final report header
        header = f"""
        # CYBERSECURITY DISINFORMATION DETECTION BRIEFING

        **Target Outlet:** {state['outlet']}
        **Country/Region:** {state['country']}
        **Source URL:** {state['url']}
        **Analysis Date:** {briefing_data['analysis_date']}
        **Articles Analyzed:** {briefing_data['articles_analyzed']}
        **System Status:** {'✓ Complete' if briefing_data['errors_encountered'] == 0 else f'⚠ {briefing_data["errors_encountered"]} errors'}

        ---

        ## EXECUTIVE SUMMARY

        This briefing analyzes {briefing_data['articles_analyzed']} recent articles from {state['outlet']} for potential disinformation patterns, propaganda techniques, and cybersecurity implications.

        ---

        ## DETAILED ANALYSIS

        """

        # Combine all sections
        state["summaries"] = summaries
        state["final_report"] = header + "\n\n---\n\n".join(summaries)

        # Add error summary if needed
        if state.get("errors"):
            error_section = "\n\n---\n\n## SYSTEM ERRORS\n\n"
            error_section += "\n".join([f"• {error}" for error in state["errors"]])
            state["final_report"] += error_section

        config.logger.info("✓ Intelligence briefing generated successfully")

    except Exception as e:
        error_msg = f"Critical error generating briefing: {str(e)}"
        config.logger.error(error_msg)
        state["errors"].append(error_msg)
        state["final_report"] = f"# BRIEFING GENERATION FAILED\n\nError: {error_msg}"

    return state

# **7. WORKFLOW DEFINITION**

In [20]:
def create_briefing_workflow():
    """Create the enhanced briefing workflow"""

    # Create the workflow graph
    workflow = StateGraph(BriefingState)

    # Add enhanced nodes
    workflow.add_node("fetch", fetch_articles)
    workflow.add_node("analyze", analyze_articles)
    workflow.add_node("digest", digest_articles)
    workflow.add_node("translate", translate_content)
    workflow.add_node("summarize", generate_intelligence_brief)

    # Define the flow
    workflow.set_entry_point("fetch")
    workflow.add_edge("fetch", "analyze")
    workflow.add_edge("analyze", "digest")
    workflow.add_edge("digest", "translate")
    workflow.add_edge("translate", "summarize")
    workflow.add_edge("summarize", END)

    return workflow.compile()

In [21]:
# Create the enhanced briefing bot
briefing_bot = create_briefing_workflow()

# **8. TESTING AND VALIDATION FUNCTIONS**

In [22]:
def validate_system():
    """Validate system components"""
    config.logger.info("Validating CDDBS system components...")

    validation_results = {
        "api_keys": False,
        "gemini_client": False,
        "workflow": False,
        "database": False
    }

    # Check API keys
    if config.gemini_api_key and config.serper_api_key:
        validation_results["api_keys"] = True
        config.logger.info("✓ API keys configured")
    else:
        config.logger.error("✗ API keys missing")

    # Check Gemini client
    try:
        test_response = config.client.models.generate_content(
            model=config.default_model,
            contents="Say 'System test successful'"
        )
        if test_response.text:
            validation_results["gemini_client"] = True
            config.logger.info("✓ Gemini client working")
    except Exception as e:
        config.logger.error(f"✗ Gemini client failed: {e}")

    # Check workflow
    if briefing_bot:
        validation_results["workflow"] = True
        config.logger.info("✓ Workflow compiled")
    else:
        config.logger.error("✗ Workflow compilation failed")

    # Check database (optional)
    try:
        if db_manager.create_tables():
            validation_results["database"] = True
            config.logger.info("✓ Database connection working")
    except:
        config.logger.warning("⚠ Database not available (optional)")

    return validation_results

def run_test_analysis(outlet="BBC", url="bbc.com", country="UK"):
    """Run a test analysis"""
    config.logger.info("Running test analysis...")

    start_time = time.time()

    test_state = {
        "outlet": outlet,
        "url": url,
        "country": country,
        "articles": [],
        "summaries": [],
        "final_report": "",
        "errors": [],
        "processing_time": 0.0
    }

    try:
        result = briefing_bot.invoke(test_state)
        result["processing_time"] = time.time() - start_time

        config.logger.info(f"✓ Test completed in {result['processing_time']:.2f} seconds")
        return result

    except Exception as e:
        config.logger.error(f"✗ Test failed: {e}")
        return None

# 9. MAIN EXECUTION INTERFACE

In [24]:
def run_cddbs_analysis(outlet: str, url: str, country: str, num_articles: int = 3):
    """Main interface for running CDDBS analysis"""

    print(f"""
    🎯 CDDBS Analysis Starting
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Outlet: {outlet}
    URL: {url}
    Country: {country}
    Articles: {num_articles}
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """)

    start_time = time.time()

    # Initialize state
    state = {
        "outlet": outlet,
        "url": url,
        "country": country,
        "articles": [],
        "summaries": [],
        "final_report": "",
        "errors": [],
        "processing_time": 0.0
    }

    try:
        # Run the analysis
        result = briefing_bot.invoke(state)
        result["processing_time"] = time.time() - start_time

        # Display results
        print(f"""
        ✅ Analysis Complete
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        Processing Time: {result['processing_time']:.2f}s
        Articles Processed: {len(result['articles'])}
        Errors: {len(result['errors'])}
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        """)

        return result

    except Exception as e:
        print(f"❌ Analysis Failed: {e}")
        return None

In [25]:
# =============================================================================
# 10. QUICK START EXAMPLES
# =============================================================================

def quick_start_examples():
    """Show quick start examples"""

    examples = [
        {"outlet": "RT", "url": "rt.com", "country": "Russia"},
        {"outlet": "Xinhua", "url": "english.news.cn", "country": "China"},
        {"outlet": "Press TV", "url": "presstv.ir", "country": "Iran"},
        {"outlet": "BBC", "url": "bbc.com", "country": "UK"},
        {"outlet": "CNN", "url": "cnn.com", "country": "USA"}
    ]

    print("🚀 CDDBS Quick Start Examples:")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

    for i, example in enumerate(examples, 1):
        print(f"{i}. {example['outlet']} ({example['country']})")
        print(f"   result = run_cddbs_analysis('{example['outlet']}', '{example['url']}', '{example['country']}')")

    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print("💡 Example: result = run_cddbs_analysis('RT', 'rt.com', 'Russia')")
    print("📊 View results: print(result['final_report'])")

# =============================================================================
# SYSTEM INITIALIZATION
# =============================================================================

print("""
==============================================
    CDDBS - Cybersecurity Disinformation
    Detection Briefing System v2.0
==============================================
""")

# Validate system on startup
validation_results = validate_system()

if all([validation_results["api_keys"], validation_results["gemini_client"], validation_results["workflow"]]):
    print("🎉 CDDBS System Ready!")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    quick_start_examples()
else:
    print("❌ System validation failed. Please check configuration.")

# Display workflow visualization if available
try:
    display(Image(briefing_bot.get_graph().draw_mermaid_png()))
except:
    print("📊 Workflow visualization not available in this environment")

print("\n🔧 System ready for analysis. Use run_cddbs_analysis() to begin.")


    CDDBS - Cybersecurity Disinformation
    Detection Briefing System v2.0



ERROR:CDDBS:Database connection failed: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



🎉 CDDBS System Ready!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🚀 CDDBS Quick Start Examples:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. RT (Russia)
   result = run_cddbs_analysis('RT', 'rt.com', 'Russia')
2. Xinhua (China)
   result = run_cddbs_analysis('Xinhua', 'english.news.cn', 'China')
3. Press TV (Iran)
   result = run_cddbs_analysis('Press TV', 'presstv.ir', 'Iran')
4. BBC (UK)
   result = run_cddbs_analysis('BBC', 'bbc.com', 'UK')
5. CNN (USA)
   result = run_cddbs_analysis('CNN', 'cnn.com', 'USA')
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
💡 Example: result = run_cddbs_analysis('RT', 'rt.com', 'Russia')
📊 View results: print(result['final_report'])
📊 Workflow visualization not available in this environment

🔧 System ready for analysis. Use run_cddbs_analysis() to begin.


In [26]:
run_cddbs_analysis('RT', 'rt.com', 'Russia')


    🎯 CDDBS Analysis Starting
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Outlet: RT
    URL: rt.com
    Country: Russia
    Articles: 3
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    

        ✅ Analysis Complete
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        Processing Time: 243.56s
        Articles Processed: 5
        Errors: 0
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        


{'outlet': 'RT',
 'url': 'rt.com',
 'country': 'Russia',
 'articles': [{'title': 'Musk reacts to Telegram founder’s Moldova election meddling claim',
   'link': 'https://www.rt.com/news/625471-musk-telegram-moldova-election-meddling/',
   'snippet': '',
   'date': '09/28/2025, 01:31 PM, +0000 UTC',
   'source': {'name': 'rt.com',
    'icon': 'https://encrypted-tbn3.gstatic.com/faviconV2?url=https://www.rt.com&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
   'meta': {'position': 1,
    'title': 'Musk reacts to Telegram founder’s Moldova election meddling claim',
    'source': {'name': 'rt.com',
     'icon': 'https://encrypted-tbn3.gstatic.com/faviconV2?url=https://www.rt.com&client=NEWS_360&size=96&type=FAVICON&fallback_opts=TYPE,SIZE,URL'},
    'link': 'https://www.rt.com/news/625471-musk-telegram-moldova-election-meddling/',
    'thumbnail': 'https://mf.b37mrtl.ru/files/2025.09/article/68d9384885f5404ff737d83c.jpg',
    'thumbnail_small': 'https://news.google.com